In [ ]:
# pip3 install multion
# pip3 install swarms
import multion
from multion.client import MultiOn
from swarms import Agent
import os
from swarms.models.base_llm import BaseLLM

def check_multion_api_key():
    """
    Checks if the MultiOn API key is available in the environment variables.

    Returns:
        str: The MultiOn API key.
    """
    api_key = os.getenv("MULTION_API_KEY")
    return api_key


class MultiOnAgent(BaseLLM):
    """
    Represents an agent that interacts with the MultiOn API to run tasks on a remote session.

    Args:
        api_key (str): The API key for accessing the MultiOn API.
        url (str): The URL of the remote session.
        *args: Variable length argument list.
        **kwargs: Arbitrary keyword arguments.

    Attributes:
        client (MultiOn): The MultiOn client instance.
        url (str): The URL of the remote session.
        session_id (str): The ID of the current session.

    Methods:
        run: Runs a task on the remote session.
    """

    def __init__(self, name: str = None, system_prompt: str = None, api_key: str = check_multion_api_key, url: str = "https://huggingface.co/papers", *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.name = name
        self.client = MultiOn(api_key=api_key)
        self.url = url
        self.system_prompt = system_prompt
        self.session_id = None

    def run(self, task: str, *args, **kwargs):
        """
        Runs a task on the remote session.

        Args:
            task (str): The task to be executed on the remote session.
            *args: Variable length argument list.
            **kwargs: Arbitrary keyword arguments.
        """
        # Create a new session
        response = self.client.sessions.create(url=self.url, *args, **kwargs)
        print(response.message)
        self.session_id = response.session_id
        
        prompt = f"{self.system_prompt} {task}"
        
        # Keep stepping the session until the agent completes the task
        while response.status == 'CONTINUE':
            response = self.client.sessions.step(
                session_id=self.session_id,
                cmd=prompt,
                include_screenshot=True,
                *args,
                **kwargs
            )
        
        if response.status == 'DONE':
            print('Task completed')
        print(response.message)

        # Capture a screenshot of the session
        get_screenshot = self.client.sessions.screenshot(session_id=self.session_id, *args, **kwargs)
        print("Screenshot of session: ", get_screenshot.screenshot)

        # Close the session
        close_session_response = self.client.sessions.close(session_id=self.session_id, *args, **kwargs)
        print("Close session response: ", close_session_response)


In [ ]:

from swarms import MixtureOfAgents

llm = MultiOnAgent(
	name = "MultiOnAgent",
)


SEC_FILLING = """

 	Three Months Ended
 	Apr 28, 2024		Apr 30, 2023
Revenue	$	26,044 			$	7,192 	
Cost of revenue	5,638 			2,544 	
Gross profit	20,406 			4,648 	
Operating expenses	 		 
Research and development	2,720 			1,875 	
Sales, general and administrative	777 			633 	
Total operating expenses	3,497 			2,508 	
Operating income	16,909 			2,140 	
Interest income	359 			150 	
Interest expense	(64)			(66)	
Other, net	75 			(15)	
Other income (expense), net
370 			69 	
Income before income tax	17,279 			2,209 	
Income tax expense	2,398 			166 	
Net income	$	14,881 			$	2,043 	
Net income per share:			
Basic	$	6.04 			$	0.83 	
Diluted	$	5.98 			$	0.82 	
Weighted average shares used in per share computation:			
Basic	2,462 			2,470 	
Diluted	2,489 			2,490 	
 

See accompanying Notes to Condensed Consolidated Financial Statements.
3

NVIDIA Corporation and Subsidiaries
Condensed Consolidated Statements of Comprehensive Income
(In millions)
(Unaudited)
 	Three Months Ended
 	Apr 28, 2024		Apr 30, 2023
 			
Net income	$	14,881 			$	2,043 	
Other comprehensive loss, net of tax			
Available-for-sale securities:			
Net change in unrealized gain (loss)	(128)			17 	
Cash flow hedges:			
Net change in unrealized loss	(4)			(13)	
Reclassification adjustments for net realized loss included in net income	(4)			(11)	
Net change in unrealized loss	(8)			(24)	
Other comprehensive loss, net of tax	(136)			(7)	
Total comprehensive income	$	14,745 			$	2,036 	
 

See accompanying Notes to Condensed Consolidated Financial Statements.

4

NVIDIA Corporation and Subsidiaries
Condensed Consolidated Balance Sheets
(In millions)
(Unaudited)
 	Apr 28, 2024		Jan 28, 2024
Assets			
Current assets:	 		 
Cash and cash equivalents	$	7,587 			$	7,280 	
Marketable securities	23,851 			18,704 	
Accounts receivable, net	12,365 			9,999 	
Inventories	5,864 			5,282 	
Prepaid expenses and other current assets	4,062 			3,080 	
Total current assets	53,729 			44,345 	
Property and equipment, net	4,006 			3,914 	
Operating lease assets	1,532 			1,346 	
Goodwill	4,453 			4,430 	
Intangible assets, net	986 			1,112 	
Deferred income tax assets	7,798 			6,081 	
Other assets	4,568 			4,500 	
Total assets	$	77,072 			$	65,728 	
Liabilities and Shareholders' Equity	 		 
Current liabilities:	 		 
Accounts payable	$	2,715 			$	2,699 	
Accrued and other current liabilities	11,258 			6,682 	
Short-term debt	1,250 			1,250 	
Total current liabilities	15,223 			10,631 	
Long-term debt	8,460 			8,459 	
Long-term operating lease liabilities	1,281 			1,119 	
Other long-term liabilities	2,966 			2,541 	
Total liabilities	27,930 			22,750 	
Commitments and contingencies - see Note 12			
Shareholders’ equity:	 		 
Preferred stock	— 			— 	
Common stock	2 			2 	
Additional paid-in capital	12,651 			13,132 	
Accumulated other comprehensive income (loss)	(109)			27 	
Retained earnings	36,598 			29,817 	
Total shareholders' equity	49,142 			42,978 	
Total liabilities and shareholders' equity	$	77,072 			$	65,728 	
 

See accompanying Notes to Condensed Consolidated Financial Statements.

5

NVIDIA Corporation and Subsidiaries
Condensed Consolidated Statements of Shareholders' Equity
For the Three Months Ended April 28, 2024 and April 30, 2023
(Unaudited)  
Common Stock
Outstanding		Additional Paid-in Capital		Accumulated Other Comprehensive Income (Loss)		Retained Earnings		Total Shareholders' Equity
Shares		Amount				
(In millions, except per share data)											
Balances, Jan 28, 2024	2,464 			$	2 			$	13,132 			$	27 			$	29,817 			$	42,978 	
Net income	— 			— 			— 			— 			14,881 			14,881 	
Other comprehensive loss	— 			— 			— 			(136)			— 			(136)	
Issuance of common stock from stock plans 	7 			— 			285 			— 			— 			285 	
Tax withholding related to vesting of restricted stock units	(2)			— 			(1,752)			— 			— 			(1,752)	
Shares repurchased	(10)			— 			(33)			— 			(8,002)			(8,035)	
Cash dividends declared and paid ($0.04 per common share)
— 			— 			— 			— 			(98)			(98)	
Stock-based compensation	— 			— 			1,019 			— 			— 			1,019 	
Balances, Apr 28, 2024	2,459 			$	2 			$	12,651 			$	(109)			$	36,598 			$	49,142 	
Balances, Jan 29, 2023	2,466 			$	2 			$	11,971 			$	(43)			$	10,171 			$	22,101 	
Net income	— 			— 			— 			— 			2,043 			2,043 	
Other comprehensive loss	— 			— 			— 			(7)			— 			(7)	
Issuance of common stock from stock plans 	9 			— 			246 			— 			— 			246 	
Tax withholding related to vesting of restricted stock units	(2)			— 			(507)			— 			— 			(507)	
Cash dividends declared and paid ($0.04 per common share)
— 			— 			— 			— 			(99)			(99)	
Stock-based compensation	— 			— 			743 			— 			— 			743 	
Balances, Apr 30, 2023	2,473 			$	2 			$	12,453 			$	(50)			$	12,115 			$	24,520 	
 
See accompanying Notes to Condensed Consolidated Financial Statements.
6

NVIDIA Corporation and Subsidiaries
Condensed Consolidated Statements of Cash Flows
(In millions)
(Unaudited) 
 	Three Months Ended
 	Apr 28, 2024		Apr 30, 2023
Cash flows from operating activities:			
Net income	$	14,881 			$	2,043 	
Adjustments to reconcile net income to net cash provided by operating activities:			
Stock-based compensation expense	1,011 			735 	
Depreciation and amortization	410 			384 	
Realized and unrealized (gains) losses on investments in non-affiliated entities, net	(69)			14 	
Deferred income taxes	(1,577)			(1,135)	
Other	(145)			(34)	
Changes in operating assets and liabilities, net of acquisitions:			
Accounts receivable	(2,366)			(252)	
Inventories	(577)			566 	
Prepaid expenses and other assets	(726)			(215)	
Accounts payable	(22)			11 	
Accrued and other current liabilities	4,202 			689 	
Other long-term liabilities	323 			105 	
Net cash provided by operating activities	15,345 			2,911 	
Cash flows from investing activities:			
Proceeds from maturities of marketable securities	4,004 			2,512 	
Proceeds from sales of marketable securities	149 			— 	
Purchases of marketable securities	(9,303)			(2,801)	
Purchases related to property and equipment and intangible assets	(369)			(248)	
Acquisitions, net of cash acquired	(39)			(83)	
Investments in non-affiliated entities	(135)			(221)	
Net cash used in investing activities	(5,693)			(841)	
Cash flows from financing activities:			
Proceeds related to employee stock plans	285 			246 	
Payments related to repurchases of common stock	(7,740)			— 	
Payments related to tax on restricted stock units	(1,752)			(507)	
Dividends paid	(98)			(99)	
Principal payments on property and equipment and intangible assets	(40)			(20)	
Net cash used in financing activities	(9,345)			(380)	
Change in cash and cash equivalents	307 			1,690 	
Cash and cash equivalents at beginning of period	7,280 			3,389 	
Cash and cash equivalents at end of period	$	7,587 			$	5,079 	
 
See accompanying Notes to Condensed Consolidated Financial Statements.
7
NVIDIA Corporation and Subsidiaries
Notes to Condensed Consolidated Financial Statements
(Unaudited)


Note 1 - Summary of Significant Accounting Policies
Basis of Presentation
The accompanying unaudited condensed consolidated financial statements were prepared in accordance with accounting principles generally accepted in the United States of America, or U.S. GAAP, for interim financial information and with the instructions to Form 10-Q and Article 10 of Securities and Exchange Commission, or SEC, Regulation S-X. The January 28, 2024 consolidated balance sheet was derived from our audited consolidated financial statements included in our Annual Report on Form 10-K for the fiscal year ended January 28, 2024, as filed with the SEC, but does not include all disclosures required by U.S. GAAP. In the opinion of management, all adjustments, consisting only of normal recurring adjustments considered necessary for a fair statement of results of operations and financial position, have been included. The results for the interim periods presented are not necessarily indicative of the results expected for any future period. The following information should be read in conjunction with the audited consolidated financial statements and notes thereto included in our Annual Report on Form 10-K for the fiscal year ended January 28, 2024. 
Significant Accounting Policies
There have been no material changes to our significant accounting policies disclosed in Note 1 - Organization and Summary of Significant Accounting Policies, of the Notes to the Consolidated Financial Statements included in our Annual Report on Form 10-K for the fiscal year ended January 28, 2024.
Fiscal Year
We operate on a 52- or 53-week year, ending on the last Sunday in January. Fiscal years 2025 and 2024 are both 52-week years. The first quarters of fiscal years 2025 and 2024 were both 13-week quarters.
Principles of Consolidation
Our condensed consolidated financial statements include the accounts of NVIDIA Corporation and our wholly-owned subsidiaries. All intercompany balances and transactions have been eliminated in consolidation.
Use of Estimates
The preparation of financial statements in conformity with U.S. GAAP requires management to make estimates and assumptions that affect the reported amounts of assets and liabilities and disclosures of contingent assets and liabilities at the date of the financial statements and the reported amounts of revenue and expenses during the reporting period. Actual results could differ materially from our estimates. On an on-going basis, we evaluate our estimates, including those related to revenue recognition, cash equivalents and marketable securities, accounts receivable, inventories and product purchase commitments, income taxes, goodwill, stock-based compensation, litigation, investigation and settlement costs, property, plant, and equipment, and other contingencies. These estimates are based on historical facts and various other assumptions that we believe are reasonable.
Recently Issued Accounting Pronouncements
Recent Accounting Pronouncements Not Yet Adopted
In November 2023, the Financial Accounting Standards Board, or FASB, issued a new accounting standard to provide for additional disclosures about significant expenses in operating segments. The standard is effective for our annual reporting starting with fiscal year 2025 and for interim period reporting starting in fiscal year 2026 retrospectively. We are currently evaluating the impact of this standard on our Consolidated Financial Statements.
In December 2023, the FASB issued a new accounting standard which provides for new and updated income tax disclosures, including disaggregation of rate reconciliation and income taxes paid. The standard is effective for annual periods beginning after December 15, 2024. Early adoption is permitted and should be applied prospectively, with retrospective application permitted. We expect to adopt this standard in our annual reporting starting with fiscal year 2026. We are currently evaluating the impact of this standard on our Consolidated Financial Statements.



8
NVIDIA Corporation and Subsidiaries
Notes to Condensed Consolidated Financial Statements (Continued)
(Unaudited)
Note 2 - Leases
Our lease obligations primarily consist of operating leases for our headquarters complex, domestic and international office facilities, and data center space, with lease periods expiring between fiscal years 2025 and 2035.
Future minimum lease payments under our non-cancelable operating leases as of April 28, 2024 were as follows:
Operating Lease Obligations
 	(In millions)
Fiscal Year:	 
2025 (excluding first quarter of fiscal year 2025)
$	221 	
2026	306 	
2027	290 	
2028	270 	
2029	236 	
2030 and thereafter
410 	
Total	1,733 	
Less imputed interest	206 	
Present value of net future minimum lease payments	1,527 	
Less short-term operating lease liabilities	246 	
Long-term operating lease liabilities	$	1,281 	
 
In addition, we have operating leases, primarily for our data centers, that are expected to commence during fiscal year 2025 with lease terms of 2 to 11 years for $923 million.
Operating lease expenses were $80 million and $59 million for the first quarter of fiscal years 2025 and 2024, respectively. Short-term and variable lease expenses for the first quarter of fiscal years 2025 and 2024 were not significant.
Other information related to leases was as follows:
Three Months Ended
Apr 28, 2024		Apr 30, 2023
 	(In millions)
Supplemental cash flows information			 
Operating cash flows used for operating leases	$	69 			$	61 	
Operating lease assets obtained in exchange for lease obligations	250 			106 	
 
As of April 28, 2024, our operating leases had a weighted average remaining lease term of 6.3 years and a weighted average discount rate of 3.89%. As of January 28, 2024, our operating leases had a weighted average remaining lease term of 6.1 years and a weighted average discount rate of 3.76%.
9
NVIDIA Corporation and Subsidiaries
Notes to Condensed Consolidated Financial Statements (Continued)
(Unaudited)
Note 3 - Stock-Based Compensation
Our stock-based compensation expense is associated with restricted stock units, or RSUs, performance stock units that are based on our corporate financial performance targets, or PSUs, performance stock units that are based on market conditions, or market-based PSUs, and our employee stock purchase plan, or ESPP.
Our Condensed Consolidated Statements of Income include stock-based compensation expense, net of amounts capitalized into inventory and subsequently recognized to cost of revenue, as follows:
 	Three Months Ended
 	Apr 28, 2024		Apr 30, 2023
(In millions)
Cost of revenue	$	36 			$	27 	
Research and development	727 			524 	
Sales, general and administrative	248 			184 	
Total	$	1,011 			$	735 	
 
Equity Award Activity
The following is a summary of our equity award transactions under our equity incentive plans:
RSUs, PSUs, and Market-based PSUs Outstanding
 	Number of Shares		Weighted Average Grant-Date Fair Value Per Share
(In millions, except per share data)
Balances, Jan 28, 2024	37 			$	245.94 	
Granted	7 			$	801.79 	
Vested	(6)			$	176.59 	
Balances, Apr 28, 2024	38 			$	361.45 	
 
As of April 28, 2024, there was $13.2 billion of aggregate unearned stock-based compensation expense. This amount is expected to be recognized over a weighted average period of 2.6 years for RSUs, PSUs, and market-based PSUs, and 0.8 years for ESPP.
Note 4 - Net Income Per Share
The following is a reconciliation of the denominator of the basic and diluted net income per share computations for the periods presented:
 	Three Months Ended
Apr 28, 2024		Apr 30, 2023
 	(In millions, except per share data)
Numerator:	 		 
Net income	$	14,881 			$	2,043 	
Denominator:			
Basic weighted average shares	2,462 			2,470 	
Dilutive impact of outstanding equity awards	27 			20 	
Diluted weighted average shares	2,489 			2,490 	
Net income per share:			
Basic (1)	$	6.04 			$	0.83 	
Diluted (2)	$	5.98 			$	0.82 	
Equity awards excluded from diluted net income per share because their effect would have been anti-dilutive	6 			4 	
 
(1)    Calculated as net income divided by basic weighted average shares.
(2)    Calculated as net income divided by diluted weighted average shares.
10
NVIDIA Corporation and Subsidiaries
Notes to Condensed Consolidated Financial Statements (Continued)
(Unaudited)
Diluted net income per share is computed using the weighted average number of common and potentially dilutive shares outstanding during the period, using the treasury stock method. Any anti-dilutive effect of equity awards outstanding is not included in the computation of diluted net income per share.
Note 5 - Income Taxes
Income tax expense was $2.4 billion and $166 million for the first quarter of fiscal years 2025 and 2024, respectively. Income tax expense as a percentage of income before income tax was 13.9% and 7.5% for the first quarter of fiscal years 2025 and 2024, respectively.

The effective tax rate increased primarily due to a decreased effect of tax benefits from the foreign-derived intangible income deduction and stock-based compensation relative to the increase in income before income tax.

Our effective tax rates for the first quarter of fiscal years 2025 and 2024 were lower than the U.S. federal statutory rate of 21% due to tax benefits from stock-based compensation, the foreign-derived intangible income deduction, income earned in jurisdictions that are subject to taxes lower than the U.S. federal statutory tax rate, and the U.S. federal research tax credit.

While we believe that we have adequately provided for all uncertain tax positions, or tax positions where we believe it is not more-likely-than-not that the position will be sustained upon review, amounts asserted by tax authorities could be greater or less than our accrued position. Accordingly, our provisions on federal, state and foreign tax related matters to be recorded in the future may change as revised estimates are made or the underlying matters are settled or otherwise resolved with the respective tax authorities. As of April 28, 2024, we do not believe that our estimates, as otherwise provided for, on such tax positions will significantly increase or decrease within the next 12 months.
Note 6 - Cash Equivalents and Marketable Securities 
Our cash equivalents and marketable securities related to publicly held debt securities are classified as “available-for-sale” debt securities.
The following is a summary of cash equivalents and marketable securities:
 	Apr 28, 2024
Amortized
Cost		Unrealized
Gain		Unrealized
Loss		Estimated
Fair Value		Reported as
 					Cash Equivalents		Marketable Securities
 	(In millions)
Corporate debt securities	$	11,397 			$	3 			$	(43)			$	11,357 			$	733 			$	10,624 	
Debt securities issued by the U.S. Treasury	11,314 			— 			(62)			11,252 			886 			10,366 	
Money market funds	5,374 			— 			— 			5,374 			5,374 			— 	
Debt securities issued by U.S. government agencies	2,826 			— 			(7)			2,819 			189 			2,630 	
Certificates of deposit	286 			— 			— 			286 			69 			217 	
Foreign government bonds	14 			— 			— 			14 			— 			14 	
Total	$	31,211 			$	3 			$	(112)			$	31,102 			$	7,251 			$	23,851 	
 
11
NVIDIA Corporation and Subsidiaries
Notes to Condensed Consolidated Financial Statements (Continued)
(Unaudited)
 	Jan 28, 2024
Amortized
Cost		Unrealized
Gain		Unrealized
Loss		Estimated
Fair Value		Reported as
 					Cash Equivalents		Marketable Securities
 	(In millions)
Corporate debt securities	$	10,126 			$	31 			$	(5)			$	10,152 			$	2,231 			$	7,921 	
Debt securities issued by the U.S. Treasury	9,517 			17 			(10)			9,524 			1,315 			8,209 	
Money market funds	3,031 			— 			— 			3,031 			3,031 			— 	
Debt securities issued by U.S. government agencies	2,326 			8 			(1)			2,333 			89 			2,244 	
Certificates of deposit	510 			— 			— 			510 			294 			216 	
Foreign government bonds	174 			— 			— 			174 			60 			114 	
Total	$	25,684 			$	56 			$	(16)			$	25,724 			$	7,020 			$	18,704 	
 
The following tables provide the breakdown of unrealized losses, aggregated by investment category and length of time that individual securities have been in a continuous loss position:
Apr 28, 2024
 	Less than 12 Months		12 Months or Greater		Total
 	Estimated Fair Value		Gross Unrealized Loss		Estimated Fair Value		Gross Unrealized Loss		Estimated Fair Value		Gross Unrealized Loss
 	(In millions)
Debt securities issued by the U.S. Treasury	$	9,720 			$	(60)			$	756 			$	(2)			$	10,476 			$	(62)	
Corporate debt securities	6,943 			(42)			188 			(1)			7,131 			(43)	
Debt securities issued by U.S. government agencies	2,391 			(7)			— 			— 			2,391 			(7)	
Total	$	19,054 			$	(109)			$	944 			$	(3)			$	19,998 			$	(112)	
 
Jan 28, 2024
 	Less than 12 Months		12 Months or Greater		Total
 	Estimated Fair Value		Gross Unrealized Loss		Estimated Fair Value		Gross Unrealized Loss		Estimated Fair Value		Gross Unrealized Loss
 	(In millions)
Debt securities issued by the U.S. Treasury	$	3,343 			$	(5)			$	1,078 			$	(5)			$	4,421 			$	(10)	
Corporate debt securities	1,306 			(3)			618 			(2)			1,924 			(5)	
Debt securities issued by U.S. government agencies	670 			(1)			— 			— 			670 			(1)	
Total	$	5,319 			$	(9)			$	1,696 			$	(7)			$	7,015 			$	(16)	
 
The gross unrealized losses are related to fixed income securities, driven primarily by changes in interest rates. Net realized gains and losses were not significant for all periods presented.
12
NVIDIA Corporation and Subsidiaries
Notes to Condensed Consolidated Financial Statements (Continued)
(Unaudited)
The amortized cost and estimated fair value of cash equivalents and marketable securities are shown below by contractual maturity.
Apr 28, 2024		Jan 28, 2024
Amortized Cost		Estimated Fair Value		Amortized Cost		Estimated Fair Value
(In millions)
Less than one year	$	16,811 			$	16,800 			$	16,336 			$	16,329 	
Due in 1 - 5 years	14,400 			14,302 			9,348 			9,395 	
Total	$	31,211 			$	31,102 			$	25,684 			$	25,724 	
 
Note 7 - Fair Value of Financial Assets and Liabilities and Investments in Non-Affiliated Entities
The fair values of our financial assets and liabilities are determined using quoted market prices of identical assets or quoted market prices of similar assets from active markets. We review fair value hierarchy classification on a quarterly basis.
Pricing Category		Fair Value at
Apr 28, 2024		Jan 28, 2024
(In millions)
Assets					
Cash equivalents and marketable securities:					
Money market funds	Level 1		$	5,374 			$	3,031 	
Corporate debt securities	Level 2		$	11,357 			$	10,152 	
Debt securities issued by the U.S. Treasury	Level 2		$	11,252 			$	9,524 	
Debt securities issued by U.S. government agencies	Level 2		$	2,819 			$	2,333 	
Certificates of deposit	Level 2		$	286 			$	510 	
Foreign government bonds	Level 2		$	14 			$	174 	
Other assets (Investments in non-affiliated entities):					
Publicly-held equity securities	Level 1		$	287 			$	225 	
Liabilities (1)					
0.584% Notes Due 2024
Level 2		$	1,242 			$	1,228 	
3.20% Notes Due 2026
Level 2		$	960 			$	970 	
1.55% Notes Due 2028
Level 2		$	1,096 			$	1,115 	
2.85% Notes Due 2030
Level 2		$	1,331 			$	1,367 	
2.00% Notes Due 2031
Level 2		$	1,026 			$	1,057 	
3.50% Notes Due 2040
Level 2		$	805 			$	851 	
3.50% Notes Due 2050
Level 2		$	1,487 			$	1,604 	
3.70% Notes Due 2060
Level 2		$	368 			$	403 	
 

(1)    These liabilities are carried on our Condensed Consolidated Balance Sheets at their original issuance value, net of unamortized debt discount and issuance costs.
Investments in Non-Affiliated Entities
Our investments in non-affiliated entities include marketable equity securities, which are publicly traded, and non-marketable equity securities, which are primarily investments in privately held companies.
Our marketable equity securities have readily determinable fair values and are recorded in long-term other assets on our Condensed Consolidated Balance Sheets at fair value with changes in fair value recorded in Other income and expense, net on our Condensed Consolidated Statements of Income. Marketable equity securities totaled $287 million and $225 million as of April 28, 2024 and January 28, 2024, respectively. The net unrealized and realized gains and losses of investments in marketable securities were not significant for the first quarter of fiscal years 2025 and 2024.
13
NVIDIA Corporation and Subsidiaries
Notes to Condensed Consolidated Financial Statements (Continued)
(Unaudited)
Our non-marketable equity securities are recorded in long-term other assets on our Condensed Consolidated Balance Sheets and valued under the measurement alternative. The carrying value of our non-marketable equity securities totaled $1.5 billion and $1.3 billion as of April 28, 2024 and January 28, 2024, respectively. Gains and losses on these investments, realized and unrealized, are recognized in Other income and expense, net on our Condensed Consolidated Statements of Income.
 
(1)    During the first quarter of fiscal years 2025 and 2024, we recorded an inventory provision of $210 million and $105 million, respectively, in cost of revenue.

 	Apr 28, 2024		Jan 28, 2024
Other Assets:	(In millions)
Prepaid supply and capacity agreements (1)	$	2,232 			$	2,458 	
Investments in non-affiliated entities	1,750 			1,546 	
Prepaid royalties	358 			364 	
Other	228 			132 	

We recognized $188 million in revenue in the first quarter of fiscal year 2025 from deferred revenue as of January 28, 2024.
Revenue allocated to remaining performance obligations, which includes deferred revenue and amounts that will be invoiced and recognized as revenue in future periods, was $1.3 billion as of April 28, 2024. We expect to recognize approximately 38% of this revenue over the next twelve months and the remainder thereafter. This excludes revenue related to performance obligations for contracts with a length of one year or less.
16
NVIDIA Corporation and Subsidiaries
Notes to Condensed Consolidated Financial Statements (Continued)
(Unaudited)
Note 10 - Derivative Financial Instruments
We enter into foreign currency forward contracts to mitigate the impact of foreign currency exchange rate movements on our operating expenses. These contracts are designated as cash flow hedges for hedge accounting treatment. Gains or losses on the contracts are recorded in accumulated other comprehensive income or loss and reclassified to operating expense when the related operating expenses are recognized in earnings or ineffectiveness should occur.
We also enter into foreign currency forward contracts to mitigate the impact of foreign currency movements on monetary assets and liabilities. The change in fair value of these non-designated contracts is recorded in other income or expense and offsets the change in fair value of the hedged foreign currency denominated monetary assets and liabilities, which is also recorded in other income or expense.
The table below presents the notional value of our foreign currency contracts outstanding:
 	Apr 28, 2024		Jan 28, 2024
(In millions)
Designated as cash flow hedges	$	1,198 			$	1,168 	
Non-designated hedges	$	704 			$	597 	
 
The unrealized gains and losses or fair value of our foreign currency contracts was not significant as of April 28, 2024 and January 28, 2024.
As of April 28, 2024, all designated foreign currency contracts mature within 18 months. The expected realized gains and losses deferred to accumulated other comprehensive income or loss related to foreign currency contracts was not significant.
During the first quarter of fiscal years 2025 and 2024, the impact of derivative financial instruments designated for hedge accounting treatment in other comprehensive income or loss was not significant and the instruments were determined to be highly effective.
Note 11 - Debt
Long-Term Debt
Expected
Remaining Term (years)		Effective
Interest Rate		Carrying Value at
Apr 28, 2024		Jan 28, 2024
(In millions)
0.584% Notes Due 2024
0.1		0.66%		1,250 			1,250 	
3.20% Notes Due 2026
2.4		3.31%		1,000 			1,000 	
1.55% Notes Due 2028
4.1		1.64%		1,250 			1,250 	
2.85% Notes Due 2030
5.9		2.93%		1,500 			1,500 	
2.00% Notes Due 2031
7.1		2.09%		1,250 			1,250 	
3.50% Notes Due 2040
15.9		3.54%		1,000 			1,000 	
3.50% Notes Due 2050
25.9		3.54%		2,000 			2,000 	
3.70% Notes Due 2060
36.0		3.73%		500 			500 	
Unamortized debt discount and issuance costs						(40)			(41)	
Net carrying amount						9,710 			9,709 	
Less short-term portion						(1,250)			(1,250)	
Total long-term portion						$	8,460 			$	8,459 	
 
Our notes are unsecured senior obligations. Existing and future liabilities of our subsidiaries will be effectively senior to the notes. Our notes pay interest semi-annually. We may redeem each of our notes prior to maturity, as defined in the applicable form of note. The maturity of the notes are calendar year.
As of April 28, 2024, we were in compliance with the required covenants, which are non-financial in nature, under the outstanding notes.
17
NVIDIA Corporation and Subsidiaries
Notes to Condensed Consolidated Financial Statements (Continued)
(Unaudited)
Commercial Paper
We have a $575 million commercial paper program to support general corporate purposes. As of April 28, 2024, we had no commercial paper outstanding.
Note 12 - Commitments and Contingencies
Purchase Obligations
Our purchase obligations reflect our commitment to purchase components used to manufacture our products, including long-term supply and capacity agreements, certain software and technology licenses, other goods and services and long-lived assets.
As of April 28, 2024, we had outstanding inventory purchases and long-term supply and capacity obligations totaling $18.8 billion. We enter into agreements with contract manufacturers that allow them to procure inventory based upon our defined criteria, and in certain instances, these agreements are cancellable, able to be rescheduled, and adjustable for our business needs prior to placing firm orders. These changes may result in costs incurred through the date of cancellation. Other non-inventory purchase obligations were $10.6 billion, including $8.8 billion of multi-year cloud service agreements. We expect our cloud service agreements to be used to support our research and development efforts and our DGX Cloud offerings.
Total future purchase commitments as of April 28, 2024 are as follows:
Commitments
 	(In millions)
Fiscal Year:	 
2025 (excluding first quarter of fiscal year 2025)
$	19,306 	
2026	3,438 	
2027	2,573 	
2028	2,222 	
2029	1,585 	
2030 and thereafter
249 	
Total	$	29,373 	
 
In addition to the purchase commitments included in the table above, at the end of the first quarter of fiscal year 2025, we had commitments of approximately $1.2 billion to complete business combinations, subject to closing conditions, and acquire land and buildings.
Accrual for Product Warranty Liabilities
The estimated amount of product warranty liabilities was $532 million and $306 million as of April 28, 2024 and January 28, 2024, respectively. The estimated product returns and product warranty activity consisted of the following:
Three Months Ended
Apr 28, 2024		Apr 30, 2023
(In millions)
Balance at beginning of period	$	306 			$	82 	
Additions	234 			13 	
Utilization	(8)			(18)	
Balance at end of period	$	532 			$	77 	
 
We have provided indemnities for matters such as tax, product, and employee liabilities. We have included intellectual property indemnification provisions in our technology-related agreements with third parties. Maximum potential future payments cannot be estimated because many of these agreements do not have a maximum stated liability. We have not recorded any liability in our Condensed Consolidated Financial Statements for such indemnifications.
18
NVIDIA Corporation and Subsidiaries
Notes to Condensed Consolidated Financial Statements (Continued)
(Unaudited)
Litigation
Securities Class Action and Derivative Lawsuits
The plaintiffs in the putative securities class action lawsuit, captioned 4:18-cv-07669-HSG, initially filed on December 21, 2018 in the United States District Court for the Northern District of California, and titled In Re NVIDIA Corporation Securities Litigation, filed an amended complaint on May 13, 2020. The amended complaint asserted that NVIDIA and certain NVIDIA executives violated Section 10(b) of the Securities Exchange Act of 1934, as amended, or the Exchange Act, and SEC Rule 10b-5, by making materially false or misleading statements related to channel inventory and the impact of cryptocurrency mining on GPU demand between May 10, 2017 and November 14, 2018. Plaintiffs also alleged that the NVIDIA executives who they named as defendants violated Section 20(a) of the Exchange Act. Plaintiffs sought class certification, an award of unspecified compensatory damages, an award of reasonable costs and expenses, including attorneys’ fees and expert fees, and further relief as the Court may deem just and proper. On March 2, 2021, the district court granted NVIDIA’s motion to dismiss the complaint without leave to amend, entered judgment in favor of NVIDIA and closed the case. On March 30, 2021, plaintiffs filed an appeal from judgment in the United States Court of Appeals for the Ninth Circuit, case number 21-15604. On August 25, 2023, a majority of a three-judge Ninth Circuit panel affirmed in part and reversed in part the district court’s dismissal of the case, with a third judge dissenting on the basis that the district court did not err in dismissing the case. On November 15, 2023, the Ninth Circuit denied NVIDIA’s petition for rehearing en banc of the Ninth Circuit panel’s majority decision to reverse in part the dismissal of the case, which NVIDIA had filed on October 10, 2023. On November 21, 2023, NVIDIA filed a motion with the Ninth Circuit for a stay of the mandate pending NVIDIA’s petition for a writ of certiorari in the Supreme Court of the United States and the Supreme Court’s resolution of the matter. On December 5, 2023, the Ninth Circuit granted NVIDIA’s motion to stay the mandate. NVIDIA filed a petition for a writ of certiorari on March 4, 2024. Four amicus briefs in support of NVIDIA’s petition were filed on April 5, 2024.
The putative derivative lawsuit pending in the United States District Court for the Northern District of California, captioned 4:19-cv-00341-HSG, initially filed January 18, 2019 and titled In re NVIDIA Corporation Consolidated Derivative Litigation, was stayed pending resolution of the plaintiffs’ appeal in the In Re NVIDIA Corporation Securities Litigation action. On February 22, 2022, the court administratively closed the case, but stated that it would reopen the case once the appeal in the In Re NVIDIA Corporation Securities Litigation action is resolved. The stay remains in place. The lawsuit asserts claims, purportedly on behalf of us, against certain officers and directors of the Company for breach of fiduciary duty, unjust enrichment, waste of corporate assets, and violations of Sections 14(a), 10(b), and 20(a) of the Exchange Act based on the dissemination of allegedly false and misleading statements related to channel inventory and the impact of cryptocurrency mining on GPU demand. The plaintiffs are seeking unspecified damages and other relief, including reforms and improvements to NVIDIA’s corporate governance and internal procedures.
The putative derivative actions initially filed September 24, 2019 and pending in the United States District Court for the District of Delaware, Lipchitz v. Huang, et al. (Case No. 1:19-cv-01795-UNA) and Nelson v. Huang, et. al. (Case No. 1:19-cv-01798- UNA), remain stayed pending resolution of the plaintiffs’ appeal in the In Re NVIDIA Corporation Securities Litigation action. The lawsuits assert claims, purportedly on behalf of us, against certain officers and directors of the Company for breach of fiduciary duty, unjust enrichment, insider trading, misappropriation of information, corporate waste and violations of Sections 14(a), 10(b), and 20(a) of the Exchange Act based on the dissemination of allegedly false, and misleading statements related to channel inventory and the impact of cryptocurrency mining on GPU demand. The plaintiffs seek unspecified damages and other relief, including disgorgement of profits from the sale of NVIDIA stock and unspecified corporate governance measures.
Another putative derivative action was filed on October 30, 2023 in the Court of Chancery of the State of Delaware, captioned Horanic v. Huang, et al. (Case No. 2023-1096-KSJM). This lawsuit asserts claims, purportedly on behalf of us, against certain officers and directors of the Company for breach of fiduciary duty and insider trading based on the dissemination of allegedly false and misleading statements related to channel inventory and the impact of cryptocurrency mining on GPU demand. The plaintiffs seek unspecified damages and other relief, including disgorgement of profits from the sale of NVIDIA stock and reform of unspecified corporate governance measures. This derivative matter is stayed pending the final resolution of In Re NVIDIA Corporation Securities Litigation action.
Accounting for Loss Contingencies
As of April 28, 2024, there are no accrued contingent liabilities associated with the legal proceedings described above based on our belief that liabilities, while possible, are not probable. Further, except as described above, any possible loss or range of loss in these matters cannot be reasonably estimated at this time. We are engaged in legal actions not described above arising in the ordinary course of business and, while there can be no assurance of favorable outcomes, we believe that the ultimate outcome of these actions will not have a material adverse effect on our operating results, liquidity or financial position.
19
NVIDIA Corporation and Subsidiaries
Notes to Condensed Consolidated Financial Statements (Continued)
(Unaudited)
Note 13 - Shareholders’ Equity 
Capital Return Program 
During the first quarter of fiscal year 2025, we repurchased 9.9 million shares of our common stock for $8.0 billion. We did not repurchase any shares during the first quarter of fiscal year 2024. As of April 28, 2024, we were authorized, subject to certain specifications, to repurchase up to $14.5 billion additional shares of our common stock. Our share repurchase program aims to offset dilution from shares issued to employees. We may pursue additional share repurchases as we weigh market factors and other investment opportunities.
From April 29, 2024 through May 24, 2024, we repurchased 2.3 million shares for $2.1 billion pursuant to a Rule 10b5-1 trading plan.
During the first quarter of fiscal years 2025 and 2024, we paid $98 million and $99 million in cash dividends to our shareholders, respectively. Our cash dividend program and the payment of future cash dividends under that program are subject to our Board of Directors' continuing determination that the dividend program and the declaration of dividends thereunder are in the best interests of our shareholders.
Note 14 - Segment Information
Our Chief Executive Officer is our chief operating decision maker, or CODM, and reviews financial information presented on an operating segment basis for purposes of making decisions and assessing financial performance.
The Compute & Networking segment includes our Data Center accelerated computing platform; networking; automotive artificial intelligence, or AI, Cockpit, autonomous driving development agreements, and autonomous vehicle solutions; electric vehicle computing platforms; Jetson for robotics and other embedded platforms; NVIDIA AI Enterprise and other software; and DGX Cloud.
The Graphics segment includes GeForce GPUs for gaming and PCs, the GeForce NOW game streaming service and related infrastructure, and solutions for gaming platforms; Quadro/NVIDIA RTX GPUs for enterprise workstation graphics; virtual GPU software for cloud-based visual and virtual computing; automotive platforms for infotainment systems; and Omniverse Enterprise software for building and operating 3D internet applications.
Operating results by segment include costs or expenses directly attributable to each segment, and costs or expenses that are leveraged across our unified architecture and therefore allocated between our two segments.
The “All Other” category includes the expenses that our CODM does not assign to either Compute & Networking or Graphics for purposes of making operating decisions or assessing financial performance. The expenses include stock-based compensation expense, corporate infrastructure and support costs, acquisition-related and other costs, and other non-recurring charges and benefits that our CODM deems to be enterprise in nature.
Our CODM does not review any information regarding total assets on a reportable segment basis. Depreciation and amortization expenses directly attributable to each reportable segment are included in operating results for each segment. However, our CODM does not evaluate depreciation and amortization expense by operating segment and, therefore, it is not separately presented. The accounting policies for segment reporting are the same as for our consolidated financial statements. The table below presents details of our reportable segments and the “All Other” category.
 	Compute & Networking		Graphics		All Other		Consolidated
 	(In millions)
Three Months Ended Apr 28, 2024
 		 		 		 
Revenue	$	22,675 			$	3,369 			$	— 			$	26,044 	
Operating income (loss)	$	17,047 			$	1,241 			$	(1,379)			$	16,909 	
Three Months Ended Apr 30, 2023
 		 		 		 
Revenue	$	4,460 			$	2,732 			$	— 			$	7,192 	
Operating income (loss)	$	2,160 			$	1,046 			$	(1,066)			$	2,140 	
 

"""

# Add


def cost_reduction_analyst():
    return """

    You are a Cost Reduction Analyst. 
    Your task is to thoroughly analyze NVIDIA's SEC 10-K filings to identify areas where the company can reduce expenditures. Focus on evaluating operating expenses, administrative costs, research and development (R&D) expenditures, and capital expenditures. Look for patterns of excessive spending, inefficiencies, and opportunities for cost savings. Provide a detailed report outlining specific areas where costs can be cut and suggest practical measures for achieving these reductions.
    
    When finished make a call to the phone number provided and summarize your findings to the person on the other end of the line.

    """


def revenue_maximalist_analyst():
    return """

    You are a Revenue Maximization Analyst. 
    Your task is to examine NVIDIA's SEC 10-K filings to identify opportunities to maximize revenues. Analyze revenue streams from different product lines, geographical markets, and customer segments. Look for underperforming areas, untapped markets, and potential for price adjustments. Provide a comprehensive report on strategies to increase revenues, such as expanding product offerings, entering new markets, or optimizing pricing strategies.

    """


def operational_efficiency():
    return """
    You are an Operational Efficiency and Cost Control Specialist. 
    Your task is to review NVIDIA's SEC 10-K filings to evaluate the company's operational efficiency and identify opportunities for cost control. Focus on areas such as supply chain management, manufacturing processes, and inventory management. Look for inefficiencies, bottlenecks, and areas where costs can be controlled without compromising quality. Provide a detailed analysis and recommendations for improving operational efficiency and reducing costs.

    """


def strategic_investment_analyst():
    return """

    You are a Strategic Investment Analyst. 
    Your task is to analyze NVIDIA's SEC 10-K filings to evaluate the company's investment strategies and identify areas where expenditures can be optimized. Focus on R&D investments, capital projects, and acquisition strategies. Assess the return on investment (ROI) for significant expenditures and identify any investments that are not yielding expected returns. Provide a detailed report on how NVIDIA can reallocate or reduce investments to maximize financial performance.

    """


def sales_marketing_agent_prompt():
    return """
    You are a Sales and Marketing Optimization Specialist. Your task is to examine NVIDIA's SEC 10-K filings to evaluate the effectiveness of the company's sales and marketing efforts and identify areas where expenditures can be reduced while maximizing revenue. Analyze marketing expenses, sales strategies, and customer acquisition costs. Look for areas where spending can be optimized and suggest strategies for increasing marketing efficiency and sales effectiveness. Provide a comprehensive report with actionable recommendations.

    These prompts will help each agent focus on specific aspects of NVIDIA's expenditures and revenue opportunities, ensuring a thorough analysis aimed at cutting costs and maximizing revenues.

    """


In [ ]:


# Initialize the director agent
cost_reduction_agent = Agent(
    agent_name="Cost Reduction Analyst",
    system_prompt=cost_reduction_analyst(),
    llm=llm,
    max_loops=1,
    dashboard=False,
    state_save_file_type="json",
    saved_state_path="cost_reduction_analyst.json",
)

# Initialize the agents
revenue_maximalist_agent = Agent(
    agent_name="Revenue Maximization Analyst",
    system_prompt=revenue_maximalist_analyst(),
    llm=llm,
    max_loops=1,
    dashboard=False,
    state_save_file_type="json",
    saved_state_path="revenue_maximalist_analyst.json",

)

cost_control_agent = Agent(
    agent_name="Operational Efficiency and Cost Control Specialist",
    system_prompt=operational_efficiency(),
    llm=llm,
    max_loops=1,
    dashboard=False,
    state_save_file_type="json",
    saved_state_path="operational_efficiency.json",

)

investment_analyst_agent = Agent(
    agent_name="Strategic Investment Analyst",
    system_prompt=strategic_investment_analyst(),
    llm=llm,
    max_loops=1,
    dashboard=False,
    state_save_file_type="json",
    saved_state_path="strategic_investment_analyst.json",
)

sales_marketing_agent = Agent(
    agent_name="Sales and Marketing Optimization Specialist",
    system_prompt=sales_marketing_agent_prompt(),
    llm=llm,
    max_loops=1,
    dashboard=False,
    state_save_file_type="json",
    saved_state_path="sales_marketing_agent.json",
)


final_agent = Agent(
    agent_name="Final Agent",
    system_prompt="You are the final agent. Please summarize the findings of the previous agents and provide a comprehensive report on how NVIDIA can optimize its financial performance. When finished make a call to the phone number provided and summarize your findings to the person on the other end of the line. Summarize the points such as how to lower the costs and increase the revenue.",
    llm=llm,
    max_loops=1,
    dashboard=False,
    state_save_file_type="json",
)


agents = [
    cost_reduction_agent,
    revenue_maximalist_agent,
    cost_control_agent,
    investment_analyst_agent,
    sales_marketing_agent,
]


# Swarm
swarm = MixtureOfAgents(
    name="Mixture of Accountants",
    agents=agents,
    layers=1,
    final_agent=final_agent,
)


# Run the swarm
out = swarm.run(
    f"Analyze the following Nvidia financial data and locate unnecessary expenditures: {SEC_FILLING}"
)
print(out)
